# Game Log Sanitization

Strip out any personally-identifiable information from game log dump so data can be shared with the community.

In [1]:
import Pkg; 
Pkg.add("CSV");
Pkg.add("DataFrames");
Pkg.add("DataFramesMeta");

using CSV;
using DataFrames;
using DataFramesMeta;
import Dates;

    Updating registry at `/opt/julia/registries/General`
   Resolving package versions...
   Installed WeakRefStrings ────────────── v1.4.1
   Installed PooledArrays ──────────────── v1.4.0
   Installed Tables ────────────────────── v1.6.0
   Installed DataAPI ───────────────────── v1.9.0
   Installed SentinelArrays ────────────── v1.3.8
   Installed FilePathsBase ─────────────── v0.9.17
   Installed TranscodingStreams ────────── v0.9.6
   Installed DataValueInterfaces ───────── v1.0.0
   Installed CodecZlib ─────────────────── v0.7.0
   Installed InlineStrings ─────────────── v1.1.0
   Installed IteratorInterfaceExtensions ─ v1.0.0
   Installed TableTraits ───────────────── v1.0.1
   Installed CSV ───────────────────────── v0.9.11
    Updating `/opt/julia/environments/v1.6/Project.toml`
  [336ed68f] + CSV v0.9.11
    Updating `/opt/julia/environments/v1.6/Manifest.toml`
  [336ed68f] + CSV v0.9.11
  [944b1d66] + CodecZlib v0.7.0
  [9a962f9c] + DataAPI v1.9.0
  [e2d170a0] + DataValueInt

## Load the data

In [7]:
df = DataFrame(CSV.File("data.csv"));
#df = dropmissing(df, :"runner-username")
#df = dropmissing(df, :"corp-username")
size(df)

(111112, 9)

## Compute game length

In [8]:
fmt = Dates.DateFormat("yyyymmddTHHMMSSZ")
start_dt = Dates.DateTime.(df[:, :start], fmt)
end_dt = Dates.DateTime.(df[:, :end], fmt)
game_time = round.(end_dt - start_dt, Dates.Minute)
mins = map(x -> x.value, game_time)
insertcols!(df, 3, :time => mins)
size(df)

(111112, 10)

## Remove game time of day

In [9]:
transform!(df, :start => ByRow(x -> split(x, "T")[1]) => :date)
size(df)

(111112, 11)

## Hash Usernames

In [10]:
#using SHA
#using Random

#salt = randstring(5)
#runners = df[:, :"runner-username"] .* salt
#runners = bytes2hex.(sha256.(runners[:]))
#corps = df[:, :"corp-username"] .* salt
#corps = bytes2hex.(sha256.(corps[:]))
#insertcols!(df, 4, :corpusernamehash => corps)
#insertcols!(df, 5, :runnerusernamehash => runners)
#size(df)

## Write new CSV file

In [11]:
for_output = df[:, [:date, :time, :turn, :room, :format, :winner, :reason, :corp, :runner]]
fmt = Dates.DateFormat("yyyymmdd")
filename = string(Dates.format(Dates.now(), fmt), "_games.csv")
CSV.write(filename, for_output)

"20211208_games.csv"